In [ ]:
!pip install scikit-hts

     |████████████████████████████████| 61kB 5.4MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
!pip install --upgrade numpy
!pip install pmdarima

     |████████████████████████████████| 14.6MB 290kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


     |████████████████████████████████| 1.5MB 8.5MB/s 
     |████████████████████████████████| 8.7MB 52.5MB/s 
     |████████████████████████████████| 2.1MB 55.0MB/s 
ERROR: scikit-hts 0.5.2 has requirement statsmodels==0.10.2, but you'll have statsmodels 0.11.1 which is incompatible.
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: Cython 0.29.20
    Uninstalling Cython-0.29.20:
      Successfully uninstalled Cython-0.29.20


In [ ]:
import warnings
import pandas as pd
import numpy as np
import gc
import os
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
from sklearn.preprocessing import OrdinalEncoder
from  datetime import datetime, timedelta
import pickle

drive_path = Path('/content/drive/My Drive/')
warnings.filterwarnings("ignore")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


In [ ]:
#download data
!kaggle competitions download -c m5-forecasting-accuracy

!unzip -q sales_train_validation.csv.zip -d .
!unzip -q sales_train_evaluation.csv.zip -d .
!unzip -q sell_prices.csv.zip -d .
!unzip -q sample_submission.csv.zip -d .
!ls

  0% 0.00/163k [00:00<?, ?B/s]
100% 163k/163k [00:00<00:00, 64.2MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 32.9MB/s]
 58% 9.00M/15.5M [00:00<00:00, 19.2MB/s]
100% 15.5M/15.5M [00:00<00:00, 31.7MB/s]
 32% 5.00M/15.8M [00:00<00:00, 16.9MB/s]
100% 15.8M/15.8M [00:00<00:00, 45.5MB/s]
 35% 5.00M/14.2M [00:00<00:00, 13.8MB/s]
100% 14.2M/14.2M [00:00<00:00, 32.1MB/s]
calendar.csv			sales_train_validation.csv.zip
drive				sample_data
kaggle.json			sample_submission.csv
sales_train_evaluation.csv	sample_submission.csv.zip
sales_train_evaluation.csv.zip	sell_prices.csv
sales_train_validation.csv	sell_prices.csv.zip


In [ ]:
def transpose(column, index, day_col, dt):
    """
    Turn the row oriented time series into a column oriented one 
    """
    ts = []
    new_cols = dt[column].unique()
    
    for value in new_cols:
        value_ts = dt[dt[column] == value]
        vertical = value_ts[day_col].sum().T
        vertical.index = index
        ts.append(vertical)
    return pd.DataFrame({k: v for k, v in zip(new_cols, ts)})

In [ ]:
days_cols = [f'd_{i}' for i in range(1942-(364*4), 1942)]
cat_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
df_train_full = pd.read_csv('./sales_train_evaluation.csv', usecols=cat_cols+days_cols)
df_train_full.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_486,d_487,d_488,d_489,d_490,d_491,d_492,d_493,d_494,d_495,d_496,d_497,d_498,d_499,d_500,d_501,d_502,d_503,d_504,d_505,d_506,d_507,d_508,d_509,d_510,d_511,d_512,d_513,d_514,d_515,d_516,d_517,d_518,d_519,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,1,1,3,1,0,1,1,0,2,0,5,1,0,1,1,2,1,3,0,0,0,1,1,1,1,0,0,0,0,1,2,...,3,0,1,0,5,4,1,0,1,3,7,2,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,3,2,2,3,1,3,1,1,2,0,1,2,2,2,5,2,1,1,2,2,0,1,4,1,0,0,1,0,1,0,1,1,2,1,...,1,1,2,1,1,0,1,1,2,2,2,4,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0


In [ ]:
ids = df_train_full.id.unique()
df_train_full['cat_id']  = (df_train_full['store_id'] + '_' + df_train_full['cat_id'])
df_train_full['dept_id'] = (df_train_full['store_id'] + '_' + df_train_full['dept_id'])
df_train_full['id'] = (df_train_full['store_id'] + '_' + df_train_full['id'])

In [ ]:
calendar = reduce_mem_usage(pd.read_csv('calendar.csv'))

day_cols = [col for col in df_train_full.columns if col.startswith('d_')]
idx = [int(col.split('d_')[1]) for col in day_cols]

# will be useful later
train_date_id = pd.to_datetime(calendar[calendar.d.apply(lambda x: int(x.split('d_')[1])).isin(idx)].date)

Mem. usage decreased to  0.12 Mb (41.9% reduction)


In [ ]:
dept_ts = transpose('dept_id', train_date_id, day_cols, df_train_full)

In [ ]:
state_ts = transpose('state_id', train_date_id, day_cols, df_train_full)
store_ts = transpose('store_id', train_date_id, day_cols, df_train_full)
cat_ts = transpose('cat_id', train_date_id, day_cols, df_train_full)
dept_ts = transpose('dept_id', train_date_id, day_cols, df_train_full)
item_ts = transpose('id', train_date_id, day_cols, df_train_full)

df = pd.concat([state_ts, store_ts, cat_ts,dept_ts, item_ts], 1)
df.index.freq = 'D'
df.head()

,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,CA_1_HOBBIES,CA_1_HOUSEHOLD,CA_1_FOODS,CA_2_HOBBIES,CA_2_HOUSEHOLD,CA_2_FOODS,CA_3_HOBBIES,CA_3_HOUSEHOLD,CA_3_FOODS,CA_4_HOBBIES,CA_4_HOUSEHOLD,CA_4_FOODS,TX_1_HOBBIES,TX_1_HOUSEHOLD,TX_1_FOODS,TX_2_HOBBIES,TX_2_HOUSEHOLD,TX_2_FOODS,TX_3_HOBBIES,TX_3_HOUSEHOLD,TX_3_FOODS,WI_1_HOBBIES,WI_1_HOUSEHOLD,WI_1_FOODS,WI_2_HOBBIES,WI_2_HOUSEHOLD,WI_2_FOODS,...,WI_3_FOODS_3_788_WI_3_evaluation,WI_3_FOODS_3_789_WI_3_evaluation,WI_3_FOODS_3_790_WI_3_evaluation,WI_3_FOODS_3_791_WI_3_evaluation,WI_3_FOODS_3_792_WI_3_evaluation,WI_3_FOODS_3_793_WI_3_evaluation,WI_3_FOODS_3_794_WI_3_evaluation,WI_3_FOODS_3_795_WI_3_evaluation,WI_3_FOODS_3_796_WI_3_evaluation,WI_3_FOODS_3_797_WI_3_evaluation,WI_3_FOODS_3_798_WI_3_evaluation,WI_3_FOODS_3_799_WI_3_evaluation,WI_3_FOODS_3_800_WI_3_evaluation,WI_3_FOODS_3_801_WI_3_evaluation,WI_3_FOODS_3_802_WI_3_evaluation,WI_3_FOODS_3_803_WI_3_evaluation,WI_3_FOODS_3_804_WI_3_evaluation,WI_3_FOODS_3_805_WI_3_evaluation,WI_3_FOODS_3_806_WI_3_evaluation,WI_3_FOODS_3_807_WI_3_evaluation,WI_3_FOODS_3_808_WI_3_evaluation,WI_3_FOODS_3_809_WI_3_evaluation,WI_3_FOODS_3_810_WI_3_evaluation,WI_3_FOODS_3_811_WI_3_evaluation,WI_3_FOODS_3_812_WI_3_evaluation,WI_3_FOODS_3_813_WI_3_evaluation,WI_3_FOODS_3_814_WI_3_evaluation,WI_3_FOODS_3_815_WI_3_evaluation,WI_3_FOODS_3_816_WI_3_evaluation,WI_3_FOODS_3_817_WI_3_evaluation,WI_3_FOODS_3_818_WI_3_evaluation,WI_3_FOODS_3_819_WI_3_evaluation,WI_3_FOODS_3_820_WI_3_evaluation,WI_3_FOODS_3_821_WI_3_evaluation,WI_3_FOODS_3_822_WI_3_evaluation,WI_3_FOODS_3_823_WI_3_evaluation,WI_3_FOODS_3_824_WI_3_evaluation,WI_3_FOODS_3_825_WI_3_evaluation,WI_3_FOODS_3_826_WI_3_evaluation,WI_3_FOODS_3_827_WI_3_evaluation
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012-05-28,14997,11254,7156,4070,3278,5588,2061,3344,4732,3178,1633,2010,3513,500,595,2975,266,902,2110,369,1077,4142,218,301,1542,202,603,2539,337,733,3662,215,530,2433,269,343,1021,92,376,1542,...,1,0,0,0,8,0,0,0,0,2,0,0,12,20,2,0,15,8,0,1,53,12,0,0,3,0,5,0,0,0,1,1,2,0,3,0,0,0,0,0
2012-05-29,12934,9172,6514,3277,2409,5033,2215,2808,3636,2728,1594,1804,3116,529,436,2312,237,635,1537,363,976,3694,261,235,1719,155,492,2161,273,575,2788,171,472,2085,256,368,970,116,442,1246,...,0,0,0,0,3,1,0,0,0,2,0,0,8,4,2,0,10,4,0,4,25,4,0,2,1,0,4,0,4,1,1,2,1,0,0,0,0,2,0,0
2012-05-30,12078,8441,6631,2977,2239,5055,1807,2578,3388,2475,1517,2145,2969,440,415,2122,228,564,1447,392,955,3708,212,244,1351,140,447,1991,246,677,2465,158,414,1903,254,376,887,169,495,1481,...,2,0,0,0,9,1,0,0,0,3,0,2,2,1,1,0,7,14,0,0,24,4,0,21,3,0,6,0,1,1,3,8,1,0,1,0,0,0,0,0
2012-05-31,13501,8090,7218,3546,2687,5189,2079,2258,3383,2449,1872,2487,2859,489,512,2545,285,649,1753,423,931,3835,256,315,1508,165,488,1605,297,650,2436,212,438,1799,348,432,1092,188,651,1648,...,1,0,0,0,6,1,1,0,0,0,0,0,5,4,2,0,3,6,0,3,21,2,0,9,2,0,6,0,2,3,3,2,1,0,0,0,0,1,0,0
2012-06-01,16806,12246,10477,4575,3244,6874,2113,3569,4735,3942,2421,3551,4505,496,656,3423,379,809,2056,514,1606,4754,210,302,1601,207,857,2505,448,869,3418,306,1019,2617,515,591,1315,279,1070,2202,...,0,0,0,0,14,0,0,0,0,2,0,2,12,5,1,0,9,8,0,5,24,6,0,5,7,0,4,0,9,0,0,7,2,0,0,0,0,0,0,0


In [ ]:
df['total'] = df['TX'].astype(float) + df['CA'].astype(float) + df['WI'].astype(float)

In [ ]:
df.tail()

,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,CA_1_HOBBIES,CA_1_HOUSEHOLD,CA_1_FOODS,CA_2_HOBBIES,CA_2_HOUSEHOLD,CA_2_FOODS,CA_3_HOBBIES,CA_3_HOUSEHOLD,CA_3_FOODS,CA_4_HOBBIES,CA_4_HOUSEHOLD,CA_4_FOODS,TX_1_HOBBIES,TX_1_HOUSEHOLD,TX_1_FOODS,TX_2_HOBBIES,TX_2_HOUSEHOLD,TX_2_FOODS,TX_3_HOBBIES,TX_3_HOUSEHOLD,TX_3_FOODS,WI_1_HOBBIES,WI_1_HOUSEHOLD,WI_1_FOODS,WI_2_HOBBIES,WI_2_HOUSEHOLD,WI_2_FOODS,...,WI_3_FOODS_3_789_WI_3_evaluation,WI_3_FOODS_3_790_WI_3_evaluation,WI_3_FOODS_3_791_WI_3_evaluation,WI_3_FOODS_3_792_WI_3_evaluation,WI_3_FOODS_3_793_WI_3_evaluation,WI_3_FOODS_3_794_WI_3_evaluation,WI_3_FOODS_3_795_WI_3_evaluation,WI_3_FOODS_3_796_WI_3_evaluation,WI_3_FOODS_3_797_WI_3_evaluation,WI_3_FOODS_3_798_WI_3_evaluation,WI_3_FOODS_3_799_WI_3_evaluation,WI_3_FOODS_3_800_WI_3_evaluation,WI_3_FOODS_3_801_WI_3_evaluation,WI_3_FOODS_3_802_WI_3_evaluation,WI_3_FOODS_3_803_WI_3_evaluation,WI_3_FOODS_3_804_WI_3_evaluation,WI_3_FOODS_3_805_WI_3_evaluation,WI_3_FOODS_3_806_WI_3_evaluation,WI_3_FOODS_3_807_WI_3_evaluation,WI_3_FOODS_3_808_WI_3_evaluation,WI_3_FOODS_3_809_WI_3_evaluation,WI_3_FOODS_3_810_WI_3_evaluation,WI_3_FOODS_3_811_WI_3_evaluation,WI_3_FOODS_3_812_WI_3_evaluation,WI_3_FOODS_3_813_WI_3_evaluation,WI_3_FOODS_3_814_WI_3_evaluation,WI_3_FOODS_3_815_WI_3_evaluation,WI_3_FOODS_3_816_WI_3_evaluation,WI_3_FOODS_3_817_WI_3_evaluation,WI_3_FOODS_3_818_WI_3_evaluation,WI_3_FOODS_3_819_WI_3_evaluation,WI_3_FOODS_3_820_WI_3_evaluation,WI_3_FOODS_3_821_WI_3_evaluation,WI_3_FOODS_3_822_WI_3_evaluation,WI_3_FOODS_3_823_WI_3_evaluation,WI_3_FOODS_3_824_WI_3_evaluation,WI_3_FOODS_3_825_WI_3_evaluation,WI_3_FOODS_3_826_WI_3_evaluation,WI_3_FOODS_3_827_WI_3_evaluation,total
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-05-18,15678,10375,11043,3995,3789,5375,2519,3147,3518,3710,3242,4533,3268,381,798,2816,334,894,2561,565,1385,3425,326,530,1663,257,966,1924,385,863,2270,414,894,2402,373,586,2283,373,1031,3129,...,6,0,0,2,1,0,0,0,6,0,0,6,3,0,2,8,0,0,1,0,2,1,21,1,0,7,0,0,0,3,6,3,0,0,1,0,1,0,0,37096.0
2016-05-19,16297,9162,11504,4136,4037,5580,2544,2748,3126,3288,3478,4628,3398,406,876,2854,357,955,2725,596,1528,3456,376,553,1615,229,755,1764,323,689,2114,278,805,2205,360,708,2410,284,1146,3198,...,8,2,4,1,1,1,2,0,2,0,0,8,1,1,1,5,1,1,0,0,2,0,22,0,0,2,0,0,0,1,4,3,1,2,0,1,0,1,2,36963.0
2016-05-20,17430,12303,12819,4433,4751,5542,2704,3664,4249,4390,3813,4880,4126,539,901,2993,434,1138,3179,571,1474,3497,431,588,1685,281,999,2384,442,1070,2737,409,1137,2844,438,759,2616,329,1213,3338,...,8,0,0,1,0,0,0,0,1,2,0,8,3,0,0,8,0,0,0,0,3,0,27,2,0,3,0,0,0,3,4,3,1,1,0,0,1,1,2,42552.0
2016-05-21,23103,13681,14734,5764,7120,7073,3146,4167,4802,4712,5002,5213,4519,719,1262,3783,762,1856,4502,675,2083,4315,545,712,1889,386,1177,2604,555,1062,3185,499,1168,3045,564,1043,3395,324,1282,3607,...,11,0,0,0,0,0,0,0,0,0,1,7,4,0,1,15,2,0,1,0,5,0,27,2,2,1,0,0,0,0,1,0,0,3,1,1,0,1,5,51518.0
2016-05-22,24644,14815,14879,6289,6614,8144,3597,4624,5217,4974,5054,5068,4757,646,1316,4327,660,1756,4198,754,2321,5069,454,767,2376,438,1286,2900,577,1286,3354,528,1200,3246,600,998,3456,281,1230,3557,...,8,0,3,6,2,2,3,0,0,0,0,12,2,1,1,16,0,1,1,0,1,0,31,3,1,0,0,0,1,0,1,1,4,4,1,0,2,0,1,54338.0


In [ ]:
df.to_pickle(drive_path/'df.pkl')

In [ ]:
# df = pd.read_pickle(drive_path/'df.pkl')
df.shape

(1456, 30604)

In [ ]:
states = df_train_full.state_id.unique()
stores = df_train_full.store_id.unique()
depts = df_train_full.dept_id.unique()
cats = df_train_full.cat_id.unique()
items = df_train_full.id.unique()

total = {'total': list(states)}
state_h = {k: [v for v in stores if v.startswith(k)] for k in states}
store_h = {k: [v for v in cats if v.startswith(k)] for k in stores}
cat_h = {k: [v for v in depts if v.startswith(k)] for k in cats}
dept_h = {k: [v for v in items if v.startswith(k)] for k in depts}

hierarchy = {**total, **state_h, **store_h, **cat_h, **dept_h}
gc.collect()

19428

In [ ]:
pickle.dump(hierarchy, open(drive_path/'hierarchy.json', 'wb'))

In [ ]:
# hierarchy = pickle.load(open(drive_path/'hierarchy.json', 'rb'))

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

cal = pd.read_csv("calendar.csv")[-len(df)-28:]

cal.date = pd.to_datetime(cal.date)
holiday_type_df = pd.get_dummies(cal[['event_type_1']])

one_hot_cal = cal.copy()

one_hot_cal[['event_name_1']] = one_hot_cal[['event_name_1']].fillna("None")
one_hot_cal[['event_name_1']] = OrdinalEncoder(dtype='int').fit_transform(one_hot_cal[['event_name_1']])

one_hot = pd.concat([one_hot_cal, holiday_type_df], axis=1)
one_hot_cols = ['snap_CA', 'snap_TX', 'snap_WI', 'event_name_1'] + list(holiday_type_df.columns)
one_hot = one_hot[one_hot_cols].reset_index(drop=True)

prop_holidays = pd.DataFrame()
for hol in cal['event_name_1'].dropna().unique():
    prop_holidays = pd.concat((prop_holidays, pd.DataFrame({'holiday': hol, 'ds': cal[cal['event_name_1'] == hol]['date']})))

df_ev_3 = pd.DataFrame({'holiday': 'snap_CA', 'ds': cal[cal['snap_CA'] == 1]['date']})
df_ev_4 = pd.DataFrame({'holiday': 'snap_TX', 'ds': cal[cal['snap_TX'] == 1]['date']})
df_ev_5 = pd.DataFrame({'holiday': 'snap_WI', 'ds': cal[cal['snap_WI'] == 1]['date']})
prop_holidays = pd.concat((prop_holidays, df_ev_3, df_ev_4, df_ev_5))

In [ ]:
cal.shape, df.shape, one_hot.shape

((1484, 14), (1456, 30604), (1484, 8))

In [ ]:
from pmdarima.arima import ndiffs
from pmdarima import preprocessing
import pmdarima as pm
from pmdarima.preprocessing import BoxCoxEndogTransformer
from pmdarima import pipeline
from datetime import timedelta
from pmdarima.preprocessing import FourierFeaturizer
from fbprophet import Prophet
from hts import HTSRegressor
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def create_train_exog_df():
    exog_df = pd.concat([cal[['date']][:-28].reset_index(drop=True), one_hot[:-28]], axis=1)
    return exog_df

def create_test_exog_df():
    exog_df = pd.concat([cal[['date']][-28:].reset_index(drop=True), one_hot[-28:].reset_index(drop=True)], axis=1)
    return exog_df

def create_holt_winters(col):
    y, fitted_lambda= boxcox(df[col].values + 1)
    es = ExponentialSmoothing(
        y,
        seasonal_periods=7,
        seasonal='add',
        trend='add'
    )

    m = es.fit(
        optimized=True,
        remove_bias=True,
        use_boxcox=False,
        use_basinhopping=True
    )

    f = m.predict(start=0, end=len(df)+27)
    return inv_boxcox(f,fitted_lambda) - 1

def create_prophet_model(col):
    prop_df = pd.DataFrame({'ds': df.index}, index=range(len(df.index)))
    prop_df['y'] = df[col].values

    y, fitted_lambda= boxcox(prop_df['y'].values+1)
    

    model = Prophet(
        holidays = prop_holidays,
        uncertainty_samples = False,
        changepoint_range = 0.9, 
        changepoint_prior_scale = 0.4,
        seasonality_prior_scale=7,
        holidays_prior_scale=7,
        daily_seasonality=False,
        seasonality_mode = 'multiplicative'
    )

    prop_df['y'] = y
    model.fit(prop_df)
    fo = model.make_future_dataframe(periods=28, include_history=True)
    fo = model.predict(fo)
    fo[['yhat']] = fo[['yhat']].apply(lambda x: inv_boxcox(x, fitted_lambda))
    return fo

def create_arima_pipeline(series):
    kpss_diffs = ndiffs(series, alpha=0.05, test='kpss', max_d=6)
    adf_diffs = ndiffs(series, alpha=0.05, test='adf', max_d=6)
    n_diffs_val = max(adf_diffs, kpss_diffs)

    date_feat = preprocessing.DateFeaturizer(
        column_name="date",
        with_day_of_week=True,
        with_day_of_month=True
    )

    model = pipeline.Pipeline([
        ('fourier', FourierFeaturizer(7, 3)),
        ('date', date_feat),
        ("boxcox", BoxCoxEndogTransformer(lmbda2=1)),
        ("model", pm.AutoARIMA(
            d=n_diffs_val,
            D=n_diffs_val,
            m=7,
            seasonal=False,
            max_p=2,
            max_q=2,
            stepwise=True,
            method='nm',
            maxiter=70,
            suppress_warnings=True,
            n_jobs=-1
        ))
    ])
    return model

In [ ]:
import gc
def create_arima(col):
    gc.collect()
    file_location = drive_path/f'preds/preds_valid_{col}.pkl'

    try:
        p = pickle.load(open(file_location, "rb"))
        print("Loaded complete.")

    except (OSError, IOError) as e:
        print("*" * 50)
        print(f"Fitting node {col}")
        print("*" * 50)

        series = df[col].values
        X_train_exog = create_train_exog_df()
        X_test_exog = create_test_exog_df()

        x_pipeline = create_arima_pipeline(series)
        x_pipeline.fit(series, exogenous=X_train_exog)
    
        preds = x_pipeline.predict(n_periods=28, exogenous=X_test_exog)
        in_sample_preds = x_pipeline.predict_in_sample(exogenous=X_train_exog)
        
        del x_pipeline, X_train_exog, X_test_exog, series; gc.collect()
        p = preds, in_sample_preds, col
        # pickle.dump(p, open(file_location, 'wb'))

    return p

In [ ]:
from hts.hierarchy import HierarchyTree
tree = HierarchyTree.from_nodes(nodes=hierarchy, df=df)
col_order = tree.to_pandas().columns

Index(['total', 'CA', 'TX', 'WI', 'CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1',
       'TX_2',
       ...
       'WI_3_FOODS_3_818_WI_3_evaluation', 'WI_3_FOODS_3_819_WI_3_evaluation',
       'WI_3_FOODS_3_820_WI_3_evaluation', 'WI_3_FOODS_3_821_WI_3_evaluation',
       'WI_3_FOODS_3_822_WI_3_evaluation', 'WI_3_FOODS_3_823_WI_3_evaluation',
       'WI_3_FOODS_3_824_WI_3_evaluation', 'WI_3_FOODS_3_825_WI_3_evaluation',
       'WI_3_FOODS_3_826_WI_3_evaluation', 'WI_3_FOODS_3_827_WI_3_evaluation'],
      dtype='object', length=30604)

In [ ]:
from multiprocessing import Pool, cpu_count
import pickle

print(f'Cpu count {cpu_count()}')
gc.collect()
pool = Pool(processes=cpu_count())

holt_preds = pool.map(create_holt_winters, col_order)

pool.close()
pool.join()

In [ ]:
holt_forecasts = {}

for i, col in enumerate(col_order):
    total_preds = holt_preds[i]
    pred = total_preds[-28:]
    in_sample = total_preds[:-28]
    resid = in_sample - df[col].values

    holt_forecasts[col] = {
        'y_hat': total_preds,
        'residual': resid,
        'mse': np.nanmean(np.array(resid) ** 2)
    }

pickle.dump(holt_forecasts, open(drive_path/'holt_forecasts.json', 'wb'))

In [ ]:
# holt_forecasts = pickle.load(open(drive_path/'holt_forecasts_val.json', 'rb'))

In [ ]:
from multiprocessing import Pool, cpu_count
import pickle

print(f'Cpu count {cpu_count()}')

pool = Pool(processes=cpu_count())

ar_preds = pool.map(create_arima, col_order)

pool.close()
pool.join()

Cpu count 4
**************************************************
**************************************************
Fitting node TX_1
Fitting node total
**************************************************
**************************************************
**************************************************
**************************************************
Fitting node CA_4_HOUSEHOLD
Fitting node CA_1_FOODS
**************************************************
**************************************************
**************************************************
Fitting node CA_2_HOBBIES
**************************************************
**************************************************
Fitting node TX_2
**************************************************
**************************************************
Fitting node CA_4_FOODS
**************************************************
**************************************************
Fitting node CA
**************************************************

In [ ]:
ar_forecasts = {}

for i, col in enumerate(col_order):
    pred, in_sample, _ = ar_preds[i]
    resid = (in_sample - df[col]).values
    ar_forecasts[col] = {
        'y_hat': np.concatenate([in_sample, pred]),
        'residual': resid,
        'mse': np.nanmean(np.array(resid) ** 2)
    }

pickle.dump(ar_forecasts, open(drive_path/'ar_forecasts.json', 'wb'))

In [ ]:
from multiprocessing import Pool
import pickle

pool = Pool(processes=4)

prop_preds = pool.map(create_prophet_model, col_order)

pool.close()
pool.join()

In [ ]:
prop_forecasts = {}

for i, col in enumerate(col_order):
    total_preds = prop_preds[i]['yhat'].values
    pred = total_preds[-28:]
    in_sample = total_preds[:-28]
    resid = (in_sample - df[col]).values
    prop_forecasts[col] = {
        'y_hat': prop_preds[i]['yhat'].values,
        'residual': resid,
        'mse': np.nanmean(np.array(resid) ** 2)
    }

pickle.dump(prop_forecasts, open(drive_path/'prophet_forecasts.json', 'wb'))

In [ ]:
prophet_forecasts = pickle.load(open(drive_path/'prophet_forecasts.json', 'rb'))

In [ ]:
# summing matrix
nodes = tree.level_order_traversal()
num_at_level = list(map(sum, nodes))
columns = num_at_level[-1]
bl_mat = np.identity(columns)
top = np.ones(columns)
summing_mat = bl_mat
num_levels = len(num_at_level)

for lev in range(num_levels - 1):
    summing = nodes[-(lev + 1)]
    count = 0
    num2sum_ind = 0
    B = np.zeros([num_at_level[-1]])
    for num2sum in summing:
        num2sum_ind += num2sum
        a = bl_mat[count:num2sum_ind, :]
        count += num2sum
        if np.all(B == 0):
            B = a.sum(axis=0)
        else:
            B = np.vstack((B, a.sum(axis=0)))
    summing_mat = np.vstack((B, summing_mat))
    bl_mat = B

summing_mat = np.vstack((top, summing_mat))

In [ ]:
def y_hat_matrix(forecasts, keys=None):
    if not keys:
        keys = forecasts.keys()
    first = list(forecasts.keys())[0]
    y_hat_mat = np.zeros([len(forecasts[first]['y_hat']), 1])
    for key in keys:
        f1 = np.array(forecasts[key]['y_hat'])
        f2 = f1[:, np.newaxis]
        if np.all(y_hat_mat == 0):
            y_hat_mat = f2
        else:
            y_hat_mat = np.concatenate((y_hat_mat, f2), axis=1)
    return np.nan_to_num(y_hat_mat)

def project(hat_mat: np.ndarray, sum_mat: np.ndarray, optimal_mat: np.ndarray) -> np.ndarray:
    new_mat = np.empty([hat_mat.shape[0], sum_mat.shape[0]])
    for i in range(hat_mat.shape[0]):
        new_mat[i, :] = np.dot(optimal_mat, np.transpose(hat_mat[i, :]))
    return new_mat

In [ ]:
def reconcile(forecast):
    hat_mat = y_hat_matrix(forecast)
    transposed_summing_mat = np.transpose(summing_mat)

    ols = np.dot(np.dot(summing_mat, np.linalg.inv(np.dot(transposed_summing_mat, summing_mat))), transposed_summing_mat)

    diag = [forecast[col]['mse'] for col in col_order]
    diag = np.diag(np.hstack(diag) + 0.0000001, 0)

    inner_bracket = np.dot(np.dot(transposed_summing_mat, np.linalg.inv(diag)), summing_mat)
    left = np.dot(summing_mat, np.linalg.inv(inner_bracket))
    right = np.dot(left,transposed_summing_mat)
    optimal_mat = np.dot(right, np.linalg.inv(diag))

    res = project(hat_mat=hat_mat, sum_mat=summing_mat, optimal_mat=optimal_mat)

    p_df = pd.DataFrame(res, columns=col_order)
    p_df.index = pd.date_range(df.index[-1],df.index[-1] + timedelta(days=28)).union(df.index)
    
    return p_df

In [ ]:
recon_ar = reconcile(ar_forecasts)
recon_holt = reconcile(holt_forecasts)
recon_prop = reconcile(prop_forecasts)

In [ ]:
avg_df = recon_prop.copy()

for col in avg_df.columns:
    avg_df[col] = (recon_prop[col].values*0.75 + recon_ar[col].values*0.2 + recon_holt[col].values*0.05)

avg_df.tail()

,total,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,CA_1_HOBBIES,CA_1_HOUSEHOLD,CA_1_FOODS,CA_2_HOBBIES,CA_2_HOUSEHOLD,CA_2_FOODS,CA_3_HOBBIES,CA_3_HOUSEHOLD,CA_3_FOODS,CA_4_HOBBIES,CA_4_HOUSEHOLD,CA_4_FOODS,TX_1_HOBBIES,TX_1_HOUSEHOLD,TX_1_FOODS,TX_2_HOBBIES,TX_2_HOUSEHOLD,TX_2_FOODS,TX_3_HOBBIES,TX_3_HOUSEHOLD,TX_3_FOODS,WI_1_HOBBIES,WI_1_HOUSEHOLD,WI_1_FOODS,WI_2_HOBBIES,WI_2_HOUSEHOLD,...,TX_1_HOUSEHOLD_1,TX_1_HOUSEHOLD_2,TX_1_FOODS_1,TX_1_FOODS_2,TX_1_FOODS_3,TX_2_HOBBIES_1,TX_2_HOBBIES_2,TX_2_HOUSEHOLD_1,TX_2_HOUSEHOLD_2,TX_2_FOODS_1,TX_2_FOODS_2,TX_2_FOODS_3,TX_3_HOBBIES_1,TX_3_HOBBIES_2,TX_3_HOUSEHOLD_1,TX_3_HOUSEHOLD_2,TX_3_FOODS_1,TX_3_FOODS_2,TX_3_FOODS_3,WI_1_HOBBIES_1,WI_1_HOBBIES_2,WI_1_HOUSEHOLD_1,WI_1_HOUSEHOLD_2,WI_1_FOODS_1,WI_1_FOODS_2,WI_1_FOODS_3,WI_2_HOBBIES_1,WI_2_HOBBIES_2,WI_2_HOUSEHOLD_1,WI_2_HOUSEHOLD_2,WI_2_FOODS_1,WI_2_FOODS_2,WI_2_FOODS_3,WI_3_HOBBIES_1,WI_3_HOBBIES_2,WI_3_HOUSEHOLD_1,WI_3_HOUSEHOLD_2,WI_3_FOODS_1,WI_3_FOODS_2,WI_3_FOODS_3
2016-06-15,43839.954080,17546.602119,11706.475821,14586.876140,4192.878256,4621.659806,6019.414484,2712.649573,3568.997833,4101.146702,4036.331287,3958.285682,6433.888248,4194.702210,486.686708,821.706160,2884.485388,390.063494,1001.640918,3229.955393,549.308771,1572.880502,3897.225211,352.899441,559.407570,1800.342561,257.273590,862.716092,2449.008150,376.031947,816.310900,2908.803855,368.582830,823.424647,2844.323810,375.064999,663.953530,2919.267152,291.460582,1026.441891,...,672.069632,190.646460,267.247943,372.475882,1809.284325,331.292142,44.739805,632.756423,183.554477,285.741199,440.159160,2182.903496,315.288072,53.294757,656.515812,166.908834,303.676800,535.719903,2004.927107,345.015062,30.049937,501.092004,162.861526,360.264002,668.232895,1890.770255,257.009843,34.450739,882.517430,143.924461,558.103864,1433.120424,3124.761485,225.588961,30.718215,525.775324,147.861452,340.120656,673.391673,2251.245929
2016-06-16,40251.892806,17382.481772,10766.375602,12103.035433,4163.822245,4690.321029,5817.119726,2711.218772,3225.275786,3825.960184,3715.139632,3796.897793,4830.839920,3475.297720,484.297004,827.982911,2851.542330,394.137993,1014.635159,3281.547877,540.097539,1555.984212,3721.037975,362.934885,564.200722,1784.083165,262.680481,845.949338,2116.645967,375.030974,822.637608,2628.291602,363.443672,821.650046,2530.045914,379.298851,667.756913,2749.842029,286.213843,1071.309761,...,651.232270,194.717069,247.290791,279.382913,1589.972264,332.277832,42.753141,627.423160,195.214447,281.851015,361.888310,1984.552277,314.945084,48.498588,654.484042,167.166003,297.048300,422.132916,1810.864699,346.608754,32.690097,502.569595,165.187317,362.384620,562.317471,1825.139938,253.311073,32.902770,919.464183,151.845578,454.205203,818.736244,2200.374869,218.721038,29.141053,537.931261,151.979310,296.728412,440.788972,1800.007673
2016-06-17,45130.369586,19373.030232,11721.549158,14035.790196,4882.067214,5461.922096,6191.005135,2838.035787,3485.719971,4205.690328,4030.138859,4564.729797,5336.295884,4134.764514,555.418421,969.021866,3357.626927,460.705894,1223.338830,3777.877372,574.508071,1683.327593,3933.169470,364.741451,582.521398,1890.772938,286.049010,940.706924,2258.964038,417.710965,932.866536,2855.112828,403.749567,924.906117,2701.483175,475.894482,837.866557,3250.968758,326.199926,1291.629478,...,731.477373,209.229550,263.687022,291.712845,1703.564171,376.074928,41.636037,718.386750,214.479786,308.834393,370.067325,2176.211110,357.680119,46.069448,740.380416,184.525701,329.636896,418.008167,1953.838112,438.807065,37.087417,639.025534,198.841023,427.058644,660.913010,2162.997104,288.097050,38.102877,1120.379894,171.249584,507.706894,829.526573,2381.233012,286.749701,32.585439,697.296968,189.067460,341.762942,475.909106,2111.392898
2016-06-18,54743.266773,24144.293582,13912.289757,16686.683433,6142.137400,7229.099677,7522.661664,3250.394841,4239.829564,5004.108641,4668.351552,5817.877224,5969.963459,4898.842751,658.690942,1296.733560,4186.712897,590.598

In [ ]:
avg_df.to_pickle(drive_path/'avg.pkl')

In [ ]:
df_sub_ahp = pd.DataFrame()
# ahp

for col in dept_ts:
    store_id = col[:4]
    dept_id = col
    predictions = avg_df[col][-28:]

    p_df = df_train_full.loc[(df_train_full.dept_id == dept_id) & (df_train_full.store_id == store_id)].iloc[:, np.r_[0,-56:-28]]
    df_item = p_df[['id']]
    total_day_sums = p_df.sum()[1:]
    proportions = (p_df.iloc[:, 1:] / np.array(total_day_sums).reshape((1, 28))).sum(axis=1)/28
    
    for i in range(1, 29):
        df_item[f'F{i}'] = proportions * float(predictions[i-1])

    df_sub_ahp = pd.concat([df_sub_ahp, df_item])

In [ ]:
df_sub_val = df_sub_pha

In [ ]:
df_sub_val.id = df_sub_val.id.apply(lambda x: x[5:])
c = df_sub_val.copy()
c.id = c.id.str.replace("evaluation$", "validation")
df_sub_val = pd.concat([c, df_sub_val], axis=0, sort=False)
df_sub_val.fillna(0, inplace=True)

In [ ]:
df_sub_val.to_csv('submission.csv', index=False)

In [ ]:
!kaggle competitions submit m5-forecasting-accuracy -f submission.csv

100% 32.1M/32.1M [00:04<00:00, 6.83MB/s]
Successfully submitted to M5 Forecasting - Accuracy